# Establecer acceso a Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 📌 Paso 1: Instalar las librerias necesarias

In [2]:
# 📌 Paso 1: Instalar las librerias necesarias
!pip install --upgrade fsspec==2024.9.0
!pip install scikit-learn nltk spacy pandas matplotlib datasets==3.1.0
!python -m spacy download es_core_news_sm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 5.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x

# 📌 Paso 2: Cargar un conjunto de datos de sentimiento real en español

In [3]:
import pandas as pd
import requests
from io import StringIO
from datasets import Dataset

ID_DEL_ARCHIVO = '1gyN4UdZ2PTjtgeu6ZE-EBMJ1Y_0b5cI-'  # Enlace del csv guardado en drive

url = f'https://drive.google.com/uc?id={ID_DEL_ARCHIVO}'


response = requests.get(url)
response.raise_for_status()  # Lanza una excepción para errores HTTP (4xx o 5xx)
csv_content = response.content.decode('utf-8')  # Decodifica el contenido a UTF-8
dataset = pd.read_csv(StringIO(csv_content))

df = pd.DataFrame(dataset) # Transformo a dataframe el dataset
print("Primeras filas del DataFrame de Pandas:")
print(df.head())

print("\nColumnas del DataFrame:")
print(df.columns)

print(df.shape) # Tamaño del dataframe

print(len(df)) # Filas del dataframe

Primeras filas del DataFrame de Pandas:
   Unnamed: 0                                             tweets   labels
0           0  ChatGPT: Optimizing Language Models for Dialog...  neutral
1           1  Try talking with ChatGPT, our new AI system wh...     good
2           2  ChatGPT: Optimizing Language Models for Dialog...  neutral
3           3  THRILLED to share that ChatGPT, our new model ...     good
4           4  As of 2 minutes ago, @OpenAI released their ne...      bad

Columnas del DataFrame:
Index(['Unnamed: 0', 'tweets', 'labels'], dtype='object')
(219294, 3)
219294


In [4]:
df = df.drop(columns=["Unnamed: 0"]) # Elimino la columna

In [5]:
print(df.head()) # Comprobacion
print(df.columns)
print(df.shape)

                                              tweets   labels
0  ChatGPT: Optimizing Language Models for Dialog...  neutral
1  Try talking with ChatGPT, our new AI system wh...     good
2  ChatGPT: Optimizing Language Models for Dialog...  neutral
3  THRILLED to share that ChatGPT, our new model ...     good
4  As of 2 minutes ago, @OpenAI released their ne...      bad
Index(['tweets', 'labels'], dtype='object')
(219294, 2)


# 📌 Paso 3: Limpieza y preprocesamiento del texto

In [6]:
import os
print(os.cpu_count())  # Número total de núcleos de Google Colab

2


In [7]:
# 📌 Paso 3: Limpieza y preprocesamiento del texto
import spacy

nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
# 1) Cambiamos el "es" por "en" (texto en ingles)
# 2) Deshabilita análisis sintáctico y NER (Named Entity Recognition) para mejorar rendimiento

def preprocess_texts(texts):
    processed_texts = []
    for doc in nlp.pipe((text.lower() for text in texts), batch_size=1000, n_process=2): # nlp.pipe() es mucho más eficiente que procesar cada fila individualmente con apply(), ya que procesa los textos en lotes, aprovechando el paralelismo.
      yield " ".join([token.lemma_ for token in doc if not token.is_stop and token.is_alpha])  #  No almacena todos los textos en memoria intermedia

df['clean_text'] = list(preprocess_texts(df['tweets']))  # Convierte generador en lista
print(df.head())


                                              tweets   labels  \
0  ChatGPT: Optimizing Language Models for Dialog...  neutral   
1  Try talking with ChatGPT, our new AI system wh...     good   
2  ChatGPT: Optimizing Language Models for Dialog...  neutral   
3  THRILLED to share that ChatGPT, our new model ...     good   
4  As of 2 minutes ago, @OpenAI released their ne...      bad   

                                          clean_text  
0           chatgpt optimize language model dialogue  
1  try talk chatgpt new ai system optimize dialog...  
2  chatgpt optimize language model dialogue ai ma...  
3  thrilled share chatgpt new model optimize dial...  
4           minute ago release new chatgpt use right  


In [8]:
# El proceso anterior tarda 16' por lo que vamos a guardar el dataframe resultante en un csv
# Guardar el DataFrame en la carpeta "SA" dentro de "Colab Notebooks"

df.to_csv("/content/drive/My Drive/Colab_Notebooks/SA/clean_file.csv", index=False)

print("DataFrame guardado en Google Drive en la carpeta SA")

DataFrame guardado en Google Drive en la carpeta SA


In [9]:
# Importar el nuevo csv
import pandas as pd
import requests
from io import StringIO
from datasets import Dataset

ID_DEL_ARCHIVO = '18lLl4rnbUfpbXpUVKp2FsYv9Lm_OQibA'  # Enlace del csv guardado en drive

url = f'https://drive.google.com/uc?id={ID_DEL_ARCHIVO}'


response = requests.get(url)
response.raise_for_status()  # Lanza una excepción para errores HTTP (4xx o 5xx)
csv_content = response.content.decode('utf-8')  # Decodifica el contenido a UTF-8
dataset = pd.read_csv(StringIO(csv_content))

df = pd.DataFrame(dataset) # Transformo a dataframe el dataset
print("Primeras filas del DataFrame de Pandas:")
print(df.head())

print("\nColumnas del DataFrame:")
print(df.columns)

print(df.shape) # Tamaño del dataframe

Primeras filas del DataFrame de Pandas:
                                              tweets   labels  \
0  ChatGPT: Optimizing Language Models for Dialog...  neutral   
1  Try talking with ChatGPT, our new AI system wh...     good   
2  ChatGPT: Optimizing Language Models for Dialog...  neutral   
3  THRILLED to share that ChatGPT, our new model ...     good   
4  As of 2 minutes ago, @OpenAI released their ne...      bad   

                                          clean_text  
0           chatgpt optimize language model dialogue  
1  try talk chatgpt new ai system optimize dialog...  
2  chatgpt optimize language model dialogue ai ma...  
3  thrilled share chatgpt new model optimize dial...  
4           minute ago release new chatgpt use right  

Columnas del DataFrame:
Index(['tweets', 'labels', 'clean_text'], dtype='object')
(219294, 3)


In [10]:
print(df['clean_text'].isna().sum())  # Cuenta cuántos NaN hay
df = df.dropna(subset=['clean_text']) # Esto eliminará solo las filas donde clean_text es NaN, sin afectar otras columnas.

286


# 📌 Paso 4: Extracción de características (convertir texto en características TF-IDF)

In [11]:
# 📌 Paso 4: Extracción de características (convertir texto en características TF((Término Frecuencia)-IDF(Frecuencia inversa de documentos))
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer( # Crea el vectorizador TF-IDF con un máximo de 5000 palabras
    max_features=5000,  # Usa las 5000 palabras más relevantes
    min_df=5,  # Ignora palabras que aparecen en menos de 5 documentos
    max_df=0.9,  # Ignora palabras que aparecen en más del 90% de los documentos
    stop_words='english'  # Elimina palabras muy comunes en inglés
)
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(df['clean_text']) # Transforma los textos en una matriz numérica (TF-IDF)
y = df['labels'] # Extraccion de las etiquetas
print(X.shape) #  Muestra el tamaño de la matriz TF-IDF
print(X[:, :100].toarray()) # Imprime las primeras 100 columnas
print(y)


(219008, 5000)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
0         neutral
1            good
2         neutral
3            good
4             bad
           ...   
219289        bad
219290       good
219291        bad
219292        bad
219293    neutral
Name: labels, Length: 219008, dtype: object


# 📌 Paso 5: Selección de características por puntuación avg_tfidf

In [12]:
# Selección de características por puntuación avg_tfidf

import numpy as np

NTopFeatures = 100

feature_names = vectorizer.get_feature_names_out() # Devuelve una lista con las palabras clave que corresponden a las columnas de X.

# Calcular la puntuación media TF-IDF de cada palabra en todos los tweets
avg_tfidf_scores = X.mean(axis=0).A1  #  A1 convierte la matriz en un array 1D

# Ordena las características por puntuación media TF-IDF en orden descendente
top_features_indices = np.argsort(avg_tfidf_scores)[::-1][:NTopFeatures]

# Obtener el nombre de las palabras principales
top_features = [feature_names[i] for i in top_features_indices]

# Muestra las NTopFeatures palabras
print(f"Top {NTopFeatures} most significant features:")
for feature in top_features:
    print(feature)

Top 100 most significant features:
chatgpt
ai
write
ask
openai
google
like
new
good
use
chatbot
know
code
think
answer
try
question
go
thing
get
time
work
search
need
generate
people
human
way
future
create
well
tool
come
world
day
help
bot
talk
chat
learn
change
job
prompt
replace
play
year
story
model
amp
look
tell
language
want
technology
give
amazing
tech
find
great
fun
content
twitter
start
gpt
build
take
artificialintelligence
essay
game
intelligence
say
right
love
interesting
read
post
explain
article
feel
mind
make
let
response
artificial
thank
blow
tweet
text
conversation
poem
today
business
test
pretty
idea
lot
real
see
result
open


In [13]:
# Filtremos el conjunto de datos X por las 100 características principales
# Obtener los índices de las características más importantes del vocabulario
top_features_indices = [np.where(vectorizer.get_feature_names_out() == feature)[0][0] for feature in top_features]

# Filtrar la matriz X para conservar sólo las columnas correspondientes a las características más destacadas.
X_filtered = X[:, top_features_indices]

# Imprime la forma de la matriz filtrada
print("Shape of filtered X:", X_filtered.shape)
print(X_filtered[:, :100].toarray())


Shape of filtered X: (219008, 100)
[[0.09366825 0.         0.         ... 0.         0.         0.        ]
 [0.06418643 0.15351886 0.         ... 0.         0.         0.        ]
 [0.03909153 0.18699549 0.         ... 0.         0.         0.        ]
 ...
 [1.         0.         0.         ... 0.         0.         0.        ]
 [0.06987795 0.         0.         ... 0.         0.         0.        ]
 [0.10932583 0.         0.         ... 0.         0.         0.        ]]


# 📌 Paso 6: Estadística del conjunto de datos de nuevo (sin incluir la característica seleccionada)

In [14]:
# Mezcla aleatoriamente las filas del DataFrame
df = df.sample(frac = 1, random_state=42)

# Mostramos informacion del dataset
print(df.head(), df['labels'].value_counts())
print(df['labels'].unique())
print(df.describe())

                                                   tweets   labels  \
72302   everyone running #revit api requests through #...  neutral   
143132  ChatGPT and other generative ML systems repres...      bad   
138712  I never thought creativity can be automated, u...  neutral   
66074   Grammarly keknya bakal kelindes ChatGPT juga :...  neutral   
129704  Using ChatGPT at least 2-3 times a day now for...     good   

                                               clean_text  
72302                  run revit api request chatgpt like  
143132  chatgpt generative ml system represent step ch...  
138712      think creativity automate see amp try chatgpt  
66074   grammarly keknya bakal kelindes chatgpt juga f...  
129704  chatgpt time day genuinely useful stuff esp us...   labels
bad        107563
good        55985
neutral     55460
Name: count, dtype: int64
['neutral' 'bad' 'good']
         tweets  labels clean_text
count    219008  219008     219008
unique   217338       3     188703
to

# 📌 Paso 7: Entrenar y evaluar modelos ML

In [30]:
# 📌 Paso 7: Entrenar y evaluar modelos ML
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import SGDClassifier

# Dividimos el dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y) # stratify=y mantiene el balance de clases y evita problemas de rendimiento.

# Entrenamos Naïve Bayes
nb_model = MultinomialNB(alpha=0.3)
nb_model.fit(X_train, y_train)
y_pred_nb = nb_model.predict(X_test)

# Entrenamos SVM (Cambiamos el SVC() por SGDClassifier() para acelerar SVM)
# svm_model = SVC(kernel='linear', probability=True) -> Es un SVM exacto, resolviendo el problema de optimización completamente pero lento en datasets grandes
svm_model = SGDClassifier(loss="hinge", class_weight="balanced", random_state=42) # Más rápido porque usa descenso de gradiente estocástico en lugar de resolver el problema de optimización exacto.
svm_model.fit(X_train, y_train)
y_pred_svm = svm_model.predict(X_test)

# Entrenamos Random Forest
rf_model = RandomForestClassifier(
    n_estimators=100,  # Aumentamos el número de árboles a 100 para mayor estabilidad
    max_depth=12,  # Permitimos árboles más profundos para capturar relaciones complejas
    min_samples_split=2,  # Menos restricciones en las divisiones de los árboles
    min_samples_leaf=1,  # Permite hojas con menos muestras
    class_weight="balanced",  # Maneja automáticamente clases desbalanceadas
    random_state=42,
    n_jobs=-1  # Usa todos los núcleos del CPU
)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)

# 📌 Paso 8: Métricas de evaluación de la impresión

In [31]:
# 📌 Paso 8: Métricas de evaluación de la impresión
print("🔹 Naïve Bayes Results:\n", classification_report(y_test, y_pred_nb))
print("🔹 SVM Results:\n", classification_report(y_test, y_pred_svm))
print("🔹 Random Forest Results:\n", classification_report(y_test, y_pred_rf))

🔹 Naïve Bayes Results:
               precision    recall  f1-score   support

         bad       0.73      0.96      0.83     21513
        good       0.73      0.64      0.68     11197
     neutral       0.58      0.31      0.40     11092

    accuracy                           0.71     43802
   macro avg       0.68      0.63      0.64     43802
weighted avg       0.69      0.71      0.68     43802

🔹 SVM Results:
               precision    recall  f1-score   support

         bad       0.80      0.95      0.87     21513
        good       0.69      0.84      0.76     11197
     neutral       0.72      0.31      0.43     11092

    accuracy                           0.76     43802
   macro avg       0.74      0.70      0.69     43802
weighted avg       0.75      0.76      0.73     43802

🔹 Random Forest Results:
               precision    recall  f1-score   support

         bad       0.75      0.89      0.81     21513
        good       0.67      0.74      0.71     11197
     neut